In [4]:
from torchvision.utils import save_image
import pandas as pd
from PIL import Image

In [6]:
df = pd.read_csv(r"C:\Users\mouju\Desktop\film\components_analysis\df_unique.csv", header=[0])

In [1]:
data_root = r"C:\Users\mouju\Desktop\film\geomorph_data_test"
# chips_root = r"C:\Users\mouju\Desktop\film\geomorph_data_test\chips"

In [ ]:
image = Image.open(data_root + os.path.sep + file)
chip_size = 32

In [ ]:
for for index, row in df.iterrows():
    
        lat_index_start = np.round((self.base_lat - lat) / pixel_len - self.chip_size/2).astype(int)
        lat_index_end = lat_index_start + self.chip_size
        
        lng_index_start = np.round((lng - self.base_lng) / pixel_len - self.chip_size/2).astype(int)
        lng_index_end = lng_index_start + self.chip_size
        
        image = self.preloaded[:, lat_index_start:lat_index_end,lng_index_start:lng_index_end]
        
    
    save_image(data['image'], os.path.join(data_root, 'chips', f'{i:04d}.png'))